# Import Libraries and Clone Github

In [1]:
# Import Libraries
import os
import re
import time
import pandas as pd
import google.generativeai as genai

In [2]:
# Global Variables
CATEGORIES = [
        "Finance",
        "Production",
        "Reserves / Exploration / Acquisitions / Mergers / Divestments",
        "Environment / Regulatory / Geopolitics",
        "Alternative Energy / Lower Carbon",
        "Oil Price / Natural Gas Price / Gasoline Price"]
SENTIMENT_RESULTS_FILE_PATH = 'Prompt1_Bias_Check_Sentiment_Analysis_Results.csv'

# Import Data

In [3]:
# Import all cleaned data files
invest_df1 = pd.read_csv('../02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('../02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('../02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('../02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('../02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('../02_Cleaned_Data/SEC_Filings.csv')

# Merge into single df
text_df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
display(text_df.shape)
display(text_df.head())
display(text_df.tail())

(10053, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
10048,Earnings Call Q&A,EQ-338,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN
10049,Earnings Call Q&A,EQ-339,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN
10050,Earnings Call Q&A,EQ-340,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN
10051,Earnings Call Q&A,EQ-341,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN
10052,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN


In [4]:
# Drop rows google gemini will not process
rows_to_drop = ['PQ-2840736837']
index_to_drops = text_df[text_df['Unique_ID'].isin(rows_to_drop)].index
text_df.drop(index_to_drops, inplace=True)
print(index_to_drops)

Index([5379], dtype='int64')


# Find or Create Results CSV

In [5]:
# Determine if the Sentiment Analysis Results file already exists
file_exists = os.path.isfile(SENTIMENT_RESULTS_FILE_PATH)

# Print the result
if file_exists:
    print(f"The file exists in the current directory.")
else:
    print(f"The file does not exist in the current directory.")

The file exists in the current directory.


In [6]:
# Create an empty file if the file does not exist
if not file_exists:
    # Copy text df and drop the text and headline column due to size
    empty_sentiment_df = text_df.copy()
    empty_sentiment_df = empty_sentiment_df.drop(['Article Text', 'Article Headline'], axis=1)

    for category in CATEGORIES:
        empty_sentiment_df[category] = ""
        empty_sentiment_df[category] = empty_sentiment_df[category].astype('object')

    # Display results
    display(empty_sentiment_df.head())

    # Save as CSV
    empty_sentiment_df.to_csv(SENTIMENT_RESULTS_FILE_PATH, index=False)

### NOTE: If you want to re-run the sentiment analysis, delete or archive the csv to create a blank one

# Sentiment Analysis
NOTE: Google gemini **currently** has a daily query limit of 1,500 requests per day.  As we have over 10,000 documents, the code will be designed to run over multiple days and pick up where we last left off.  After the code is run, the user will still need to manually download the csv and upload to github.

**Gemini Free Rate Limits**
*   15 RPM (requests per minute)
*   1 million TPM (tokens per minute)
*   1,500 RPD (requests per day)

In [7]:
# Set up Gemini. (API key needs to be in your environment variables)
key = 'GOOGLE_API_KEY'
GOOGLE_API_KEY = os.getenv(key)
genai.configure(api_key=GOOGLE_API_KEY)

# So it doesn't block the output
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",},]

model = genai.GenerativeModel('gemini-1.5-flash-latest', safety_settings=safety_settings)

In [8]:
# Function to find the first empty row of the csv
def find_first_unique_id_with_empty_values(file_path, categories):
    """
    Finds the first unique ID where any of the specified columns have empty values in a CSV file.

    Args:
        file_path (str): The path to the CSV file.
        categories (list of str): List of column names to check for empty values.

    Returns:
        str: The first Unique_ID where any of the specified columns have empty values.
        None: If no such row is found.
    """

    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Iterate over each row to find the first unique ID with any empty values in the specified columns
    for index, row in df.iterrows():
        if row[categories].isnull().any() or (row[categories] == '').any():
            return row['Unique_ID']

    return None  # Return None if no such row is found

# Test function
unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)
print(unique_id)

IR-7


In [9]:
# Helper function to get Gemini query function inputs
def get_gemini_inputs(text_df, unique_id):
    """
    Retrieves information from the DataFrame based on the unique ID and outputs company, source, headline, and text.

    Args:
        text_df (pd.DataFrame): The DataFrame containing the text data.
        unique_id (str): The unique ID to search for.

    Returns:
        tuple: A tuple containing company, source, headline, and text.
    """
    # Find the row with the specified unique ID
    row = text_df[text_df['Unique_ID'] == unique_id]

    # Extract the required information
    company = row['Ticker'].values[0]
    source = row['Source'].values[0]
    headline = row['Article Headline'].values[0]
    text = row['Article Text'].values[0]

    return company, source, headline, text

# Test function
company, source, headline, text = get_gemini_inputs(text_df, unique_id)
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"Text:\n{text}")

Company: EOG

Source: Investment Research

Headline: EOG Resources, Inc.

Text:
Stock Report | May 06, 2024 | NYSESymbol: EOG | EOG is in the S&P 500
EOG Resources, Inc.
Recommendation Price 12-Mo. Target Price Report Currency Investment Style
BUY « « « « «
USD 130.02 (as of market close May 03, 2024) USD 150.00 USD Large-Cap Blend
Equity Analyst Stewart Glickman, CFA
GICS Sector Energy Summary One of the largest independent exploration and production companies in the world, EOG
Sub-Industry Oil and Gas Exploration and Production Resources focuses on onshore natural gas production in North America.
Key Stock Statistics (Source: CFRA, S&P Global Market Intelligence (SPGMI), Company Reports)
52-Wk Range USD 139.67 - 106.32 Oper.EPS2024E USD 11.79 Market Capitalization[B] USD 75.81 Beta 1.41
Trailing 12-Month EPS USD 11.69 Oper.EPS2025E USD 12.44 Yield [%] 2.76 3-yr Proj. EPS CAGR[%] 2
Trailing 12-Month P/E 11.12 P/E on Oper.EPS2024E 11.03 Dividend Rate/Share USD 3.64 SPGMI's Quality Rank

In [10]:
# Function to query Gemini
def query_gemini(company, source, headline, text, model):
    """
    Query Gemini to perform sentiment analysis on text from various sources about a company.

    Parameters:
    company (str): The name of the company the text is about.
    source (str): The source of the text. Valid values are "Investment Research", "ProQuest", "SEC Filings", "Earnings Call Presentations", "Earnings Call Q&A".
    headline (str): The headline or title of the text.
    text (str): The body of the text to be analyzed.
    model: The model object used to generate content and analyze the text.

    Returns:
    str: The sentiment analysis results for predefined categories in the specified format.

    The function constructs a prompt based on the source of the text and performs sentiment analysis on the given text using the provided model.
    It analyzes the content across multiple predefined categories, determining the sentiment (Positive, Neutral, Negative) for each category.
    If a category is not mentioned or relevant based on the text content, it is marked as 'Neutral'.
    The final output is summarized in a specified format.
    """

    # Source Variables
    if source == "Investment Research":
        text_source = "an analyst report"
        text_source2 = "the analyst report"
    elif source == "ProQuest":
        text_source = "a news article"
        text_source2 = "the news article"
    elif source == "SEC Filings":
        text_source = "an SEC filing"
        text_source2 = "the SEC filing"
    elif source == "Earnings Call Presentations":
        text_source = "an earnings call presentation"
        text_source2 = "the earnings call presentation"
    elif source == "Earnings Call Q&A":
        text_source = "an earnings call Q&A session"
        text_source2 = "the earnings call Q&A session"

    # Prompt
    prompt = f"""
Given the text from {text_source} about {company}, analyze the content and perform sentiment analysis across multiple predefined categories.

Sentiment options:
  - Positive
  - Neutral
  - Negative

Categories:
  - Finance
  - Production
  - Reserves / Exploration / Acquisitions / Mergers / Divestments
  - Environment / Regulatory / Geopolitics
  - Alternative Energy / Lower Carbon
  - Oil Price / Natural Gas Price / Gasoline Price

Each category should be evaluated and given a sentiment output derived from the text.
If a category is not mentioned or relevant based on the text content, mark it as 'Neutral'.

Before giving your answer, explain your reasoning and reference the article.
After going through all the categories, provide a summary in the following format:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Example Output:
- Finance - Negative
- Production - Negative
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Positive
- Environment / Regulatory / Geopolitics - Positive
- Alternative Energy / Lower Carbon - Negative
- Oil Price / Natural Gas Price / Gasoline Price - Negative

Make sure to use plain text, do not bold or bullet the output summary.

The text from {text_source2} is below:
{headline}
{text}

Remember to summarize your final answers in the following format exactly:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Make sure to use plain text and stick to the given categories and sentiment options.
DO NOT bold or bullet the output summary.
    """

    # print(prompt)
    response = model.generate_content(prompt)
    return response.text

# Test function
response = query_gemini(company, source, headline, text, model)
print(response)

Here is a breakdown of the sentiment analysis for EOG Resources, Inc. based on the provided analyst report:

**Reasoning and References**

The sentiment analysis is derived from the report's overall tone, key phrases, and specific data points. For example, mentions of "strong free cash flow," "improving unit cost performance," and "disciplined approach to upstream capex" contribute to positive sentiment in the Finance category. Conversely, concerns about "demand weakness" and "geopolitical risk premiums" lead to a negative outlook in the Oil Price / Natural Gas Price / Gasoline Price category.

**Sentiment Analysis by Category**

- **Finance - Positive:** The report highlights EOG's strong financial position with mentions of "robust free cash flow," "low debt leverage," and a "healthy dividend yield." The analyst also expects EOG to generate a "solid free cash flow surplus" and anticipates continued dividend and buyback programs. 
    - Reference: "Based on S&P Capital IQ consensus dat

In [11]:
# Function to parse text
def parse_sentiment(text, categories):
    """
    Parses a given text for specified categories and their sentiments.

    Args:
        text (str): The input text containing categories and their sentiments.
        categories (list of str): List of category names to look for in the text.

    Returns:
        dict or str: A dictionary with categories as keys and their corresponding sentiments as values,
                     or "Did not find all categories" if any sentiment is not Positive, Neutral, or Negative.
    """
    results = {}
    valid_sentiments = {"Positive", "Neutral", "Negative"}

    for category in categories:
        # Create a regex pattern to match the format "- Category - Sentiment"
        # The category name is escaped to handle any special characters
        pattern = rf"- {re.escape(category)} - (\w+)"

        # Search for the pattern in the text
        match = re.search(pattern, text)

        # If a match is found, extract the sentiment and add it to the results dictionary
        if match:
            sentiment = match.group(1)
            if sentiment not in valid_sentiments:
                return "Did not find all categories"
            results[category] = sentiment
        else:
            return "Did not find all categories"

    return results

# Test function
fail_text = """
- Finance - Positive
- Production - Neutral
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Bad
- Environment / Regulatory / Geopolitics - Neutral
- Alternative Energy / Lower Carbon - Positive
- Oil Price / Natural Gas Price / Gasoline Price - Neutral
"""

sentiment_dict = parse_sentiment(response, CATEGORIES)
fail_sentiment = parse_sentiment("fail_text", CATEGORIES)
print(sentiment_dict)
print(fail_sentiment)

{'Finance': 'Positive', 'Production': 'Positive', 'Reserves / Exploration / Acquisitions / Mergers / Divestments': 'Positive', 'Environment / Regulatory / Geopolitics': 'Neutral', 'Alternative Energy / Lower Carbon': 'Neutral', 'Oil Price / Natural Gas Price / Gasoline Price': 'Negative'}
Did not find all categories


In [12]:
# Function to update the csv
def update_csv(file_path, unique_id, sentiment_dict):
    """
    Updates the columns of a CSV file based on the unique ID and sentiment dictionary.

    Args:
        file_path (str): The path to the CSV file.
        unique_id (str): The unique ID of the row to be updated.
        sentiment_dict (dict): A dictionary with categories as keys and their corresponding sentiments as values.

    Returns:
        None
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Find the index of the row with the specified unique ID
    row_index = df[df['Unique_ID'] == unique_id].index

    # Update the columns based on the sentiment dictionary
    for category, sentiment in sentiment_dict.items():
        df.loc[row_index, category] = sentiment

    # Save the updated DataFrame back to the CSV file
    df.to_csv(file_path, index=False)
    print(f"Row with Unique_ID '{unique_id}' has been updated.")

# Test function
update_csv(SENTIMENT_RESULTS_FILE_PATH, unique_id, sentiment_dict)

Row with Unique_ID 'IR-7' has been updated.


In [13]:
start_time = time.time()
unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)
count = 0
max_tries = 5

# Iterate through the CSV using the functions
while unique_id:
    retries = 0
    success = False

    # There are multiple errors that can happen, many of them simply need another try
    while retries < max_tries and not success:
        try:
            # Get gemini inputs
            company, source, headline, text = get_gemini_inputs(text_df, unique_id)

            # Query Gemini
            response = query_gemini(company, source, headline, text, model)

            # Parse text
            sentiment_dict = parse_sentiment(response, CATEGORIES)

            # Update the csv
            update_csv(SENTIMENT_RESULTS_FILE_PATH, unique_id, sentiment_dict)

            success = True

        except Exception as e:
            retries += 1
            print(f"Error encountered: {e}. Retry {retries}/{max_tries}")

            if retries >= max_tries:
                print(f"Failed to process unique_id {unique_id} after {max_tries} attempts. Stopping.")
                # Exit both loops
                unique_id = None
                break

    if not success:
        break

    # Get an update every 10 rows
    count += 1
    if count % 10 == 0:
        elapsed_time = time.time() - start_time
        minutes, seconds = divmod(elapsed_time, 60)
        print(f"Iteration: {count}, Elapsed Time: {int(minutes)} minutes and {seconds:.2f} seconds")

    # Find the next empty row
    unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)

    # Test print statements
    # print(unique_id)
    # print(f"Company: {company}\n")
    # print(f"Source: {source}\n")
    # print(f"Headline: {headline}\n")
    # print(f"Text:\n{text}")
    # print(response)
    # print(sentiment_dict)

Row with Unique_ID 'IR-8' has been updated.
Row with Unique_ID 'IR-9' has been updated.
Row with Unique_ID 'IR-10' has been updated.
Row with Unique_ID 'IR-11' has been updated.
Row with Unique_ID 'IR-12' has been updated.
Row with Unique_ID 'IR-13' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Row with Unique_ID 'IR-14' has been updated.
Row with Unique_ID 'IR-15' has been updated.
Row with Unique_ID 'IR-16' has been updated.
Row with Unique_ID 'IR-17' has been updated.
Iteration: 10, Elapsed Time: 1 minutes and 14.79 seconds
Row with Unique_ID 'IR-18' has been updated.
Row with Unique_ID 'IR-19' has been updated.
Row with Unique_ID 'IR-20' has been updated.
Row with Unique_ID 'IR-21' has been updated.
Row with Unique_ID 'IR-22' has been updated.
Row with Unique_ID 'IR-23' has been updated.
Row with Unique_ID 'IR-24' has been updated.
Row with Unique_ID 'IR-25' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5

Row with Unique_ID 'IR-151' has been updated.
Row with Unique_ID 'IR-152' has been updated.
Row with Unique_ID 'IR-153' has been updated.
Row with Unique_ID 'IR-154' has been updated.
Row with Unique_ID 'IR-155' has been updated.
Row with Unique_ID 'IR-156' has been updated.
Row with Unique_ID 'IR-157' has been updated.
Row with Unique_ID 'IR-158' has been updated.
Iteration: 150, Elapsed Time: 18 minutes and 44.71 seconds
Row with Unique_ID 'IR-159' has been updated.
Row with Unique_ID 'IR-160' has been updated.
Row with Unique_ID 'IR-161' has been updated.
Row with Unique_ID 'IR-162' has been updated.
Row with Unique_ID 'IR-163' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Error encountered: 'str' object has no attribute 'items'. Retry 2/5
Row with Unique_ID 'IR-164' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Row with Unique_ID 'IR-165' has been updated.
Row with Unique_ID 'IR-166' has been updated.
R

Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Row with Unique_ID 'IR-294' has been updated.
Row with Unique_ID 'IR-295' has been updated.
Row with Unique_ID 'IR-296' has been updated.
Row with Unique_ID 'IR-297' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Row with Unique_ID 'IR-298' has been updated.
Iteration: 290, Elapsed Time: 80 minutes and 10.55 seconds
Row with Unique_ID 'IR-299' has been updated.
Row with Unique_ID 'IR-300' has been updated.
Row with Unique_ID 'IR-301' has been updated.
Row with Unique_ID 'IR-302' has been updated.
Row with Unique_ID 'IR-303' has been updated.
Row with Unique_ID 'IR-304' has been updated.
Row with Unique_ID 'IR-305' has been updated.
Row with Unique_ID 'IR-306' has been updated.
Row with Unique_ID 'IR-307' has been updated.
Row with Unique_ID 'IR-308' has been updated.
Iteration: 300, Elapsed Time: 81 minutes and 25.06 seconds
Row with Unique_ID 'IR-309' has been updated.
Row with U

Row with Unique_ID 'IR-440' has been updated.
Row with Unique_ID 'IR-441' has been updated.
Row with Unique_ID 'IR-442' has been updated.
Row with Unique_ID 'IR-443' has been updated.
Row with Unique_ID 'IR-444' has been updated.
Row with Unique_ID 'IR-445' has been updated.
Row with Unique_ID 'IR-446' has been updated.
Row with Unique_ID 'IR-447' has been updated.
Row with Unique_ID 'IR-448' has been updated.
Iteration: 440, Elapsed Time: 99 minutes and 24.70 seconds
Row with Unique_ID 'IR-449' has been updated.
Row with Unique_ID 'IR-450' has been updated.
Row with Unique_ID 'IR-451' has been updated.
Row with Unique_ID 'IR-452' has been updated.
Row with Unique_ID 'IR-453' has been updated.
Row with Unique_ID 'IR-454' has been updated.
Row with Unique_ID 'IR-455' has been updated.
Row with Unique_ID 'IR-456' has been updated.
Row with Unique_ID 'IR-457' has been updated.
Row with Unique_ID 'IR-458' has been updated.
Iteration: 450, Elapsed Time: 100 minutes and 36.86 seconds
Row wit

Row with Unique_ID 'IR-590' has been updated.
Row with Unique_ID 'IR-591' has been updated.
Row with Unique_ID 'IR-592' has been updated.
Row with Unique_ID 'IR-593' has been updated.
Row with Unique_ID 'IR-594' has been updated.
Row with Unique_ID 'IR-595' has been updated.
Row with Unique_ID 'IR-596' has been updated.
Row with Unique_ID 'IR-597' has been updated.
Row with Unique_ID 'IR-598' has been updated.
Iteration: 590, Elapsed Time: 118 minutes and 56.32 seconds
Row with Unique_ID 'IR-599' has been updated.
Row with Unique_ID 'IR-600' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Error encountered: 'str' object has no attribute 'items'. Retry 2/5
Row with Unique_ID 'IR-601' has been updated.
Row with Unique_ID 'IR-602' has been updated.
Row with Unique_ID 'IR-603' has been updated.
Row with Unique_ID 'IR-604' has been updated.
Row with Unique_ID 'IR-605' has been updated.
Row with Unique_ID 'IR-606' has been updated.
Row with Unique_ID 'IR

Row with Unique_ID 'IR-737' has been updated.
Row with Unique_ID 'IR-738' has been updated.
Iteration: 730, Elapsed Time: 139 minutes and 7.76 seconds
Row with Unique_ID 'IR-739' has been updated.
Row with Unique_ID 'IR-740' has been updated.
Row with Unique_ID 'IR-741' has been updated.
Row with Unique_ID 'IR-742' has been updated.
Row with Unique_ID 'IR-743' has been updated.
Row with Unique_ID 'IR-744' has been updated.
Row with Unique_ID 'IR-745' has been updated.
Row with Unique_ID 'IR-746' has been updated.
Row with Unique_ID 'IR-747' has been updated.
Row with Unique_ID 'IR-748' has been updated.
Iteration: 740, Elapsed Time: 140 minutes and 19.32 seconds
Row with Unique_ID 'IR-749' has been updated.
Row with Unique_ID 'IR-750' has been updated.
Row with Unique_ID 'IR-751' has been updated.
Row with Unique_ID 'IR-752' has been updated.
Row with Unique_ID 'IR-753' has been updated.
Row with Unique_ID 'IR-754' has been updated.
Row with Unique_ID 'IR-755' has been updated.
Row wit

Row with Unique_ID 'IR-886' has been updated.
Row with Unique_ID 'IR-887' has been updated.
Row with Unique_ID 'IR-888' has been updated.
Iteration: 880, Elapsed Time: 158 minutes and 53.06 seconds
Row with Unique_ID 'IR-889' has been updated.
Row with Unique_ID 'IR-890' has been updated.
Row with Unique_ID 'IR-891' has been updated.
Row with Unique_ID 'IR-892' has been updated.
Row with Unique_ID 'IR-893' has been updated.
Row with Unique_ID 'IR-894' has been updated.
Row with Unique_ID 'IR-895' has been updated.
Row with Unique_ID 'IR-896' has been updated.
Row with Unique_ID 'IR-897' has been updated.
Row with Unique_ID 'IR-898' has been updated.
Iteration: 890, Elapsed Time: 160 minutes and 12.01 seconds
Row with Unique_ID 'IR-899' has been updated.
Row with Unique_ID 'IR-900' has been updated.
Row with Unique_ID 'IR-901' has been updated.
Row with Unique_ID 'IR-902' has been updated.
Row with Unique_ID 'IR-903' has been updated.
Row with Unique_ID 'IR-904' has been updated.
Row wi

Row with Unique_ID 'IR-1040' has been updated.
Row with Unique_ID 'IR-1041' has been updated.
Row with Unique_ID 'IR-1042' has been updated.
Row with Unique_ID 'IR-1043' has been updated.
Row with Unique_ID 'IR-1044' has been updated.
Row with Unique_ID 'IR-1045' has been updated.
Row with Unique_ID 'IR-1046' has been updated.
Row with Unique_ID 'IR-1047' has been updated.
Row with Unique_ID 'IR-1048' has been updated.
Iteration: 1040, Elapsed Time: 180 minutes and 12.20 seconds
Row with Unique_ID 'IR-1049' has been updated.
Row with Unique_ID 'IR-1050' has been updated.
Row with Unique_ID 'IR-1051' has been updated.
Row with Unique_ID 'IR-1052' has been updated.
Row with Unique_ID 'IR-1053' has been updated.
Row with Unique_ID 'IR-1054' has been updated.
Row with Unique_ID 'IR-1055' has been updated.
Row with Unique_ID 'IR-1056' has been updated.
Row with Unique_ID 'IR-1057' has been updated.
Row with Unique_ID 'IR-1058' has been updated.
Iteration: 1050, Elapsed Time: 181 minutes and

Row with Unique_ID 'IR-1185' has been updated.
Row with Unique_ID 'IR-1186' has been updated.
Row with Unique_ID 'IR-1187' has been updated.
Row with Unique_ID 'IR-1188' has been updated.
Iteration: 1180, Elapsed Time: 199 minutes and 40.04 seconds
Row with Unique_ID 'IR-1189' has been updated.
Row with Unique_ID 'IR-1190' has been updated.
Row with Unique_ID 'IR-1191' has been updated.
Row with Unique_ID 'IR-1192' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Row with Unique_ID 'IR-1193' has been updated.
Row with Unique_ID 'IR-1194' has been updated.
Row with Unique_ID 'IR-1195' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Row with Unique_ID 'IR-1196' has been updated.
Row with Unique_ID 'IR-1197' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Row with Unique_ID 'IR-1198' has been updated.
Iteration: 1190, Elapsed Time: 201 minutes and 15.82 seconds
Row with Unique_

Row with Unique_ID 'IR-1323' has been updated.
Row with Unique_ID 'IR-1324' has been updated.
Row with Unique_ID 'IR-1325' has been updated.
Row with Unique_ID 'IR-1326' has been updated.
Row with Unique_ID 'IR-1327' has been updated.
Row with Unique_ID 'IR-1328' has been updated.
Iteration: 1320, Elapsed Time: 219 minutes and 39.01 seconds
Row with Unique_ID 'IR-1329' has been updated.
Row with Unique_ID 'IR-1330' has been updated.
Row with Unique_ID 'IR-1331' has been updated.
Row with Unique_ID 'IR-1332' has been updated.
Row with Unique_ID 'IR-1333' has been updated.
Row with Unique_ID 'IR-1334' has been updated.
Row with Unique_ID 'IR-1335' has been updated.
Row with Unique_ID 'IR-1336' has been updated.
Row with Unique_ID 'IR-1337' has been updated.
Row with Unique_ID 'IR-1338' has been updated.
Iteration: 1330, Elapsed Time: 220 minutes and 57.14 seconds
Row with Unique_ID 'IR-1339' has been updated.
Row with Unique_ID 'IR-1340' has been updated.
Row with Unique_ID 'IR-1341' has